## Add a column of families / subfamiles to our species occurrences 
- import checklists of insects from various families / subfamilies. These were acquired through the Catalogue of Life / GBIF.org
- merge checklists, keeping the key to which family / subfamily they belong
- add row to our sound files dataset (MLNS_Insects.csv) for these families
- clean up nans



### VVV import checklists of insects from various families / subfamilies and merge

In [33]:
import numpy as nm
import pandas as pd
import matplotlib.pyplot as plt
import os

## Load sound files dataset
chirps = pd.read_csv('MLSN_Insects_with_genus.csv')

## Load Checklists
cd = os.getcwd()
new_cd = os.path.join(cd, 'Species_Checklists')
os.chdir(new_cd)
dataframes = {}
for file in os.listdir(path):

    # Extract the file name without the extension
    file_name = file.split('.')[0]  
    
    # Read the file into a pandas DataFrame
    df = pd.read_csv(file, delimiter='\t')
    
    # Store the DataFrame in the dictionary with the file name as the key
    dataframes[file_name] = df

for key, df in dataframes.items():
    df['fam_or_subfam'] = key

all_checklists = pd.concat(dataframes.values(), ignore_index=True)

os.chdir(cd)

### VVV Make the scientific name column the same for both datasets and merge

In [34]:
all_checklists['scientific_name'] = all_checklists['col:scientificName']

##merge the fam_or_subfam column with the insect audio files datset

merged_df = pd.merge(chirps, all_checklists[['scientific_name', 'fam_or_subfam']].drop_duplicates(), on='scientific_name', how='left')



### VVV Clear up nans (rows without an assigned family or subfamily)

In [35]:
## Clear up nans

# I forgot to import one subfamily, Hapithinae
hapithinae = ['Orocharis saltator', 'Orocharis', 'Hapithus agitator', 'Hapithus melodius', 
                    'Orocharis gryllodes', 'Orocharis vaginalis', 'Hapithus vagus', 
                    'Orocharis luteolira', 'Orocharis diplastes', 'Orocharis nigrifrons', 
                    'Orocharis tricornis', 'Hapithus protos', 'Hapithus','Laurepa valida',
                    'Laurepa', 'Laurepa kropion', 'Antillicharis facetus']
merged_df.loc[merged_df['scientific_name'].isin(hapithinae), 'fam_or_subfam'] = 'Hapithinae'

# I didn't import a grasshopper checklist, but their weren't that many
acrididae = ['Froggattina australis', 'Dissosteira carolina', 'Romalea microptera', 'Acrididae']
merged_df.loc[merged_df['scientific_name'].isin(acrididae), 'fam_or_subfam'] = 'Acrididae'

# this is a new family...very few instances
phalangopsidae = ['Amphiacusta annulipes', 'Amphiacusta']
merged_df.loc[merged_df['scientific_name'].isin(phalangopsidae), 'fam_or_subfam'] = 'Phalangopsidae'

# unnamed gryllacrididae (leaf rolling / rasping crickets) 
merged_df.loc[merged_df['scientific_name']=='Gryllacrididae', 'fam_or_subfam'] = 'Gryllacrididae'

# place these orphans back in enopterinae. Not sure why they weren't in their rightful spot before. found their rightful spot by referencing the Orthopterist Society Table of Sp
enopterinae = ['Diatrypa sibilans', 'Paroecanthus', 'Paroecanthus hwinanus']
merged_df.loc[merged_df['scientific_name'].isin(enopterinae), 'fam_or_subfam'] = 'Eneopterinae'

# placing a couple species in this subfamily. Very few
listroscelidinae = ['Neobarrettia spinosa', 'Neobarrettia victoriae']
merged_df.loc[merged_df['scientific_name'].isin(listroscelidinae), 'fam_or_subfam'] = 'Listroscelidinae'

# get rid of Bufonidae (frogs), unspecified Gryllidae (family of crickets which is HUGE and diverse), unspecified Tettigoniidae (family of katydids, which is huge and diverse) and unspecified superfamilies
exclude = ['Gryllidae', 'Tettigoniidae', 'Bufonidae', 'Tettigonioidea','Grylloidea']
merged_df = merged_df.loc[merged_df['scientific_name'].isin(exclude)==False]



### VVV Check for nans

In [36]:
merged_df[merged_df.fam_or_subfam.isna()]

,cat_num,format,common_name,genus,scientific_name,background_species,recordist,date,year,month,...,ebird_species_code,taxon_category,taxonomic_sort,recordist_2,average_community_rating,number_of_ratings,asset_tags,original_image_height,original_image_width,fam_or_subfam


### VVV Write the CSV

In [37]:
# write csv
merged_df.to_csv('MLNS_Insects_Fams.csv', index=False)